In [51]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [52]:
velocidade = [10, 20, 30, 40, 50, 60, 70, 80]
viscosidade = [0.121, 0.139, 0.153, 0.159, 0.172, 0.172, 0.183, 0.185]
theta = math.radians(0.5)  # converte graus para radianos

var_veloc = pd.Series(velocidade)  #
var_visco = pd.Series(viscosidade)
var_freq = pd.Series(velocidade)
var_freq = var_freq * (2 * math.pi) / 60  # converte a frequência em hz para velocidade angular rad/s
var_visco = var_visco * 1000  # viscosidade a 10³
angular = var_freq / theta # taxa de deformação du/dy
angularlog = np.log10(angular)
viscolog = np.log10(var_visco)

# dicionário de dados
bloco_de_dados = {'Velocidade (RPM)': var_veloc,
                  'Vel. Angular (rad/s)': var_freq,
                  'Taxa de Deformação (ω/θ)': angular,
                  'Viscosidade (N.s/m²*10³)': var_visco,
                  'Deformação log (ln(ω/θ))': angularlog,
                  'Viscosidade log (ln(n))':viscolog
                  }

# formatação com pandas dataframe
quadro_info = pd.DataFrame(bloco_de_dados)
quadro_info

,Velocidade (RPM),Vel. Angular (rad/s),Taxa de Deformação (ω/θ),Viscosidade (N.s/m²*10³),Deformação log (ln(ω/θ)),Viscosidade log (ln(n))
0,10,1.047198,120.0,121.0,2.079181,2.082785
1,20,2.094395,240.0,139.0,2.380211,2.143015
2,30,3.141593,360.0,153.0,2.556303,2.184691
3,40,4.188790,480.0,159.0,2.681241,2.201397
4,50,5.235988,600.0,172.0,2.778151,2.235528
5,60,6.283185,720.0,172.0,2.857332,2.235528
6,70,7.330383,840.0,183.0,2.924279,2.262451
7,80,8.377580,960.0,185.0,2.982271,2.267172


In [57]:
x_sm = sm.add_constant(angular)
resultados = sm.OLS(var_visco, x_sm).fit()
resultados.summary()

C:\Users\alana\anaconda3\envs\CALCULONUMERICO\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     92.60
Date:                Sat, 09 Apr 2022   Prob (F-statistic):           7.21e-05
Time:                        23:40:47   Log-Likelihood:                -24.424
No. Observations:                   8   AIC:                             52.85
Df Residuals:                       6   BIC:                             53.01
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        120.9643      4.611     26.236      0.000     109.683     132.246
0              0.0732      0.008      9.623      0.000       0.055       0.092
==============================================================================
Omnibus:                        0.433   Durbin-Watson:                   1.265
Prob(Omnibus):                  0.805   Jarque-Bera (JB):                0.473
Skew:                          -0.327   Prob(JB):                        0.789
Kurtosis:                       2.004   Cond. No.                     1.34e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.34e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
resultados.predict(x_sm)

array([2.08253689, 2.1447802 , 2.1811902 , 2.20702351, 2.22706138,
       2.24343351, 2.25727595, 2.26926682])